In [60]:
import pandas as pd

In [61]:
from collections import defaultdict

In [62]:
import pickle as pkl

## hownet文件夹的情感词，分成正面和负面情感词

In [63]:
columns=["词语","程度级别词语（中文）.txt","负面评价词语（中文）.txt","负面情感词语（中文）.txt"
         ,"正面评价词语（中文）.txt","正面情感词语（中文）.txt","主张词语（中文）.txt"]

In [64]:
columns[1:2]

['程度级别词语（中文）.txt']

In [65]:
anls_word_dict = defaultdict(list)

#### 文本变成列表

In [66]:
for name in columns[1:]:
    cnt = 0
    file = open("./hownet/" + name, "r")
    for d in file:
        s1 = d.find("“")
        t1 = d.find("”")
        if s1 != -1 and t1 != -1:
            word_list = d[s1:t1].split("|")
        else:
            word_list = d.strip().split()
        if word_list == []:
            continue
        else:
            anls_word_dict[name.split(".")[0]].extend(word_list)
#             print(word)

In [67]:
anls_word_dict.keys()

dict_keys(['程度级别词语（中文）', '负面评价词语（中文）', '负面情感词语（中文）', '正面评价词语（中文）', '正面情感词语（中文）', '主张词语（中文）'])

In [68]:
anls_word_dict['正面情感词语（中文）'] = set(anls_word_dict['正面评价词语（中文）']) | set(anls_word_dict['正面情感词语（中文）'])
anls_word_dict['负面情感词语（中文）'] = set(anls_word_dict['负面评价词语（中文）']) | set(anls_word_dict['负面情感词语（中文）'])
word_list = set(anls_word_dict['负面情感词语（中文）']) | set(anls_word_dict['正面情感词语（中文）'])
if '负面评价词语（中文）' in anls_word_dict:
    del anls_word_dict['负面评价词语（中文）']
if '正面评价词语（中文）' in anls_word_dict:
    del anls_word_dict['正面评价词语（中文）']
if '程度级别词语（中文）' in anls_word_dict:
    del anls_word_dict['程度级别词语（中文）']
if '主张词语（中文）' in anls_word_dict:
    del anls_word_dict['主张词语（中文）']

In [69]:
anls_word_dict.keys()

dict_keys(['负面情感词语（中文）', '正面情感词语（中文）'])

##### 情感词作为dataframe的index

In [70]:
word = set(anls_word_dict['正面情感词语（中文）']) | set(anls_word_dict['负面情感词语（中文）'])

In [71]:
anls_word_table = pd.DataFrame(index=word,columns=list(anls_word_dict.keys())).fillna(0)

In [72]:
for key in anls_word_dict:
    if key == '正面情感词语（中文）':
        anls_word_table.ix[anls_word_dict[key], [key]] = 1
    elif key == '负面情感词语（中文）':
        anls_word_table.ix[anls_word_dict[key], [key]] = 1

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """


In [73]:
anls_word_table['中性情感词语（中文）'] = 0

In [74]:
anls_word_table.head()

,负面情感词语（中文）,正面情感词语（中文）,中性情感词语（中文）
满目疮痍,1,0,0
冷销,1,0,0
谒,0,1,0
烦,1,0,0
浪荡,1,0,0


## 使用训练集的情感词，来扩充词典

In [75]:
file = open("./clean_data/reference_emotion.pkl","rb")
data_remain=pkl.load(file)
file.close()

In [76]:
for i,themes,emotions,anls in data_remain:
    for j, a_emotion in enumerate(emotions):
        anl = int(anls[j])
        if  a_emotion not in anls_word_table.index:
            anls_word_table.ix[a_emotion,:]=0
        if anl == 1:
            anls_word_table.ix[a_emotion, ['正面情感词语（中文）']]=1
        elif anl == -1:
            anls_word_table.ix[a_emotion, ['负面情感词语（中文）']]=1
        else:
            anls_word_table.ix[a_emotion, ['中性情感词语（中文）']]=1
            anls_word_dict['中性情感词语（中文）'].append(a_emotion)

In [77]:
anls_word_table['中性情感词语（中文）'].unique()

array([ 0.,  1.])

In [78]:
anls_word_dict.keys()

dict_keys(['负面情感词语（中文）', '正面情感词语（中文）', '中性情感词语（中文）'])

In [79]:
anls_word_table.to_csv("anls_word_table.csv")

In [80]:
anls_word_table.tail()

,负面情感词语（中文）,正面情感词语（中文）,中性情感词语（中文）
不易裂开,0.0,1.0,0.0
容易干,1.0,0.0,0.0
没保障,1.0,0.0,0.0
不是均匀,1.0,0.0,0.0
不香,1.0,0.0,0.0


In [81]:
file = open("anls_word_dict.pkl","wb")
pkl.dump(anls_word_dict, file)
file.close()

In [82]:
['NULL']*3

['NULL', 'NULL', 'NULL']